<a href="https://colab.research.google.com/github/Techdevweb/AutoKeras/blob/main/AutoKeras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install autokeras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 162 kB 5.3 MB/s 
     |████████████████████████████████| 135 kB 45.3 MB/s 
     |████████████████████████████████| 1.6 MB 50.6 MB/s 


In [6]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.datasets import fetch_california_housing

import autokeras as ak

In [7]:
house_dataset = fetch_california_housing()
df = pd.DataFrame(
    np.concatenate(
        (house_dataset.data, house_dataset.target.reshape(-1, 1)), axis=1
    ),
    columns=house_dataset.feature_names + ["Price"],
)
train_size = int(df.shape[0] * 0.9)
df[:train_size].to_csv("train.csv", index=False)
df[train_size:].to_csv("eval.csv", index=False)
train_file_path = "train.csv"
test_file_path = "eval.csv"

The second step is to run the StructuredDataRegressor. As a quick demo, we set epochs to 10. You can also leave the epochs unspecified for an adaptive number of epochs.




In [8]:
# Initialize the structured data regressor.
reg = ak.StructuredDataRegressor(
    overwrite=True, max_trials=3
)  # It tries 3 different models.
# Feed the structured data regressor with training data.
reg.fit(
    # The path to the train.csv file.
    train_file_path,
    # The name of the label column.
    "Price",
    epochs=10,
)
# Predict with the best model.
predicted_y = reg.predict(test_file_path)
# Evaluate the best model with testing data.
print(reg.evaluate(test_file_path, "Price"))


Trial 3 Complete [00h 00m 30s]
val_loss: 1.5738937854766846

Best val_loss So Far: 0.8822022676467896
Total elapsed time: 00h 01m 23s
Epoch 1/10
581/581 [==============================] - 2s 2ms/step - loss: 1.1125 - mean_squared_error: 1.1125
Epoch 2/10
581/581 [==============================] - 1s 2ms/step - loss: 0.6259 - mean_squared_error: 0.6259
Epoch 3/10
581/581 [==============================] - 1s 2ms/step - loss: 0.5521 - mean_squared_error: 0.5521
Epoch 4/10
581/581 [==============================] - 1s 2ms/step - loss: 0.5310 - mean_squared_error: 0.5310
Epoch 5/10
581/581 [==============================] - 2s 3ms/step - loss: 0.5185 - mean_squared_error: 0.5185
Epoch 6/10
581/581 [==============================] - 1s 2ms/step - loss: 0.5086 - mean_squared_error: 0.5086
Epoch 7/10
581/581 [==============================] - 1s 2ms/step - loss: 0.5013 - mean_squared_error: 0.5013
Epoch 8/10
581/581 [==============================] - 1s 2ms/step - loss: 0.4929 - mean_squared_

Data Format

In [9]:
# x_train as pandas.DataFrame, y_train as pandas.Series
x_train = pd.read_csv(train_file_path)
print(type(x_train))  # pandas.DataFrame
y_train = x_train.pop("Price")
print(type(y_train))  # pandas.Series

# You can also use pandas.DataFrame for y_train.
y_train = pd.DataFrame(y_train)
print(type(y_train))  # pandas.DataFrame

# You can also use numpy.ndarray for x_train and y_train.
x_train = x_train.to_numpy()
y_train = y_train.to_numpy()
print(type(x_train))  # numpy.ndarray
print(type(y_train))  # numpy.ndarray

# Preparing testing data.
x_test = pd.read_csv(test_file_path)
y_test = x_test.pop("Price")

# It tries 10 different models.
reg = ak.StructuredDataRegressor(max_trials=3, overwrite=True)
# Feed the structured data regressor with training data.
reg.fit(x_train, y_train, epochs=10)
# Predict with the best model.
predicted_y = reg.predict(x_test)
# Evaluate the best model with testing data.
print(reg.evaluate(x_test, y_test))


Trial 3 Complete [00h 00m 20s]
val_loss: 1.0395352840423584

Best val_loss So Far: 0.9117365479469299
Total elapsed time: 00h 01m 07s
Epoch 1/10
581/581 [==============================] - 2s 2ms/step - loss: 1.2315 - mean_squared_error: 1.2315
Epoch 2/10
581/581 [==============================] - 1s 2ms/step - loss: 0.6262 - mean_squared_error: 0.6262
Epoch 3/10
581/581 [==============================] - 1s 2ms/step - loss: 0.5549 - mean_squared_error: 0.5549
Epoch 4/10
581/581 [==============================] - 1s 2ms/step - loss: 0.5320 - mean_squared_error: 0.5320
Epoch 5/10
581/581 [==============================] - 1s 2ms/step - loss: 0.5200 - mean_squared_error: 0.5200
Epoch 6/10
581/581 [==============================] - 1s 2ms/step - loss: 0.5069 - mean_squared_error: 0.5069
Epoch 7/10
581/581 [==============================] - 1s 2ms/step - loss: 0.4982 - mean_squared_error: 0.4982
Epoch 8/10
581/581 [==============================] - 1s 2ms/step - loss: 0.4921 - mean_squared_

convert numpy.ndarray to tf.data.Dataset.

In [10]:
train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test))

reg = ak.StructuredDataRegressor(max_trials=3, overwrite=True)
# Feed the tensorflow Dataset to the regressor.
reg.fit(train_set, epochs=10)
# Predict with the best model.
predicted_y = reg.predict(test_set)
# Evaluate the best model with testing data.
print(reg.evaluate(test_set))


Trial 3 Complete [00h 00m 37s]
val_loss: 1.1260777711868286

Best val_loss So Far: 0.9189949631690979
Total elapsed time: 00h 02m 00s
Epoch 1/10
581/581 [==============================] - 3s 4ms/step - loss: 1.2488 - mean_squared_error: 1.2488
Epoch 2/10
581/581 [==============================] - 2s 4ms/step - loss: 0.6109 - mean_squared_error: 0.6109
Epoch 3/10
581/581 [==============================] - 2s 3ms/step - loss: 0.5530 - mean_squared_error: 0.5530
Epoch 4/10
581/581 [==============================] - 2s 3ms/step - loss: 0.5352 - mean_squared_error: 0.5352
Epoch 5/10
581/581 [==============================] - 2s 3ms/step - loss: 0.5258 - mean_squared_error: 0.5258
Epoch 6/10
581/581 [==============================] - 2s 3ms/step - loss: 0.5177 - mean_squared_error: 0.5177
Epoch 7/10
581/581 [==============================] - 2s 3ms/step - loss: 0.5108 - mean_squared_error: 0.5108
Epoch 8/10
581/581 [==============================] - 2s 3ms/step - loss: 0.5061 - mean_squared_

You can also specify the column names and types for the data as follows

In [11]:
# Initialize the structured data regressor.
reg = ak.StructuredDataRegressor(
    column_names=[
        "MedInc",
        "HouseAge",
        "AveRooms",
        "AveBedrms",
        "Population",
        "AveOccup",
        "Latitude",
        "Longitude",
    ],
    column_types={"MedInc": "numerical", "Latitude": "numerical"},
    max_trials=10,  # It tries 10 different models.
    overwrite=True,
)


Validation of Data

By default, AutoKeras use the last 20% of training data as validation data. As shown in the example below, you can use validation_split to specify the percentage.

In [12]:
reg.fit(
    x_train,
    y_train,
    # Split the training data and use the last 15% as validation data.
    validation_split=0.15,
    epochs=10,
)


Trial 10 Complete [00h 00m 28s]
val_loss: 0.8608582019805908

Best val_loss So Far: 0.8243456482887268
Total elapsed time: 00h 04m 37s
Epoch 1/10
581/581 [==============================] - 2s 2ms/step - loss: 1.2686 - mean_squared_error: 1.2686
Epoch 2/10
581/581 [==============================] - 1s 3ms/step - loss: 0.6345 - mean_squared_error: 0.6345
Epoch 3/10
581/581 [==============================] - 1s 2ms/step - loss: 0.5744 - mean_squared_error: 0.5744
Epoch 4/10
581/581 [==============================] - 1s 3ms/step - loss: 0.5441 - mean_squared_error: 0.5441
Epoch 5/10
581/581 [==============================] - 1s 2ms/step - loss: 0.5285 - mean_squared_error: 0.5285
Epoch 6/10
581/581 [==============================] - 1s 2ms/step - loss: 0.5182 - mean_squared_error: 0.5182
Epoch 7/10
581/581 [==============================] - 1s 2ms/step - loss: 0.5117 - mean_squared_error: 0.5117
Epoch 8/10
581/581 [==============================] - 1s 2ms/step - loss: 0.5059 - mean_squared

In [13]:
model = reg.export_model()
model.summary()
# numpy array in object (mixed type) is not supported.
# you need convert it to unicode or float first.
model.predict(x_train)


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 8)]               0         
                                                                 
 multi_category_encoding (Mu  (None, 8)                0         
 ltiCategoryEncoding)                                            
                                                                 
 normalization (Normalizatio  (None, 8)                17        
 n)                                                              
                                                                 
 dense (Dense)               (None, 32)                288       
                                                                 
 re_lu (ReLU)                (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 32)                1056  

array([[4.70792  ],
       [5.000969 ],
       [4.408681 ],
       ...,
       [1.2937664],
       [1.0701051],
       [1.6324224]], dtype=float32)